# Testing NOBIL data

### connecting with duckdb

In [ ]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.csmd_table_dump;")
    df_csmd = cursor.df()


df_csmd.head()


In [ ]:
for kommun in df_csmd["municipality"].unique():
    print(kommun)

In [ ]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.status_online_table_dump;")
    df_st = cursor.df()
    
df_st.head()

In [ ]:
with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    cursor = conn.execute("SELECT * FROM staging.connector_table_dump;")
    df_conn = cursor.df()
    
df_conn

In [ ]:
from datetime import date
import datetime

x = datetime.datetime.now()

today_date = date(x.year, x.month, x.day).isoformat()

print(today_date)


In [ ]:
import requests 
from constants.utils import API_KEY_NOBIL


params = {
    "apikey": API_KEY_NOBIL,
    "countrycode": "SWE",
    "format": "json",
    "file": "false",
}

url = "https://nobil.no/api/server/datadump.php"

r = requests.get(url=url, params=params)
r.raise_for_status()

In [ ]:
data = r.json()

In [ ]:
data["chargerstations"][0][""]

In [ ]:
from pathlib import Path
import pandas as pd
path = Path.cwd().parents[0]/"data"



df = pd.read_excel(path/"trafikanalys.xlsx", sheet_name= "Data")





In [ ]:
df.head()

In [ ]:

df = df.reset_index().rename({"index": "id"}, axis=1)
df["id"] = df["id"].apply(lambda x: x+1)




In [ ]:
df 

In [ ]:

df = df.to_dict(orient= "records")



In [ ]:
df

# Trafikanalys filter kommun och län


In [ ]:
import duckdb
from pathlib import Path


path_duckdb = Path.cwd().parent / "data_warehouse"

with duckdb.connect(path_duckdb /"data.duckdb") as conn:
    
    df= conn.execute("select * from staging.trafikanalys;")
    
    df_trafikanalys = df.df()
    
    

In [ ]:

# ~df_trafikanalys, "~" betyder Not, så det som inte är lika med det i contains kommer det att printas ut
# astype för sätta data typ, str vektoresering för att använda strip på varje post, str med contains är vektoresera varje post och använder
# contains med regex, case är boolesk om det ska vara case sensitive och regex boolesk för tolka det man skickar in är regular expression. 
filter = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]



In [ ]:

kommuner = sorted(filter["kommun"].dropna().unique().tolist())






In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)


print(kommuner)

In [ ]:

# Samma som för kommun fast nu är det för län
df_filter_län = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]





In [ ]:
df_filter_län

In [ ]:
län = sorted(df_filter_län["l_n"].dropna().unique().tolist())

In [ ]:
print(län)

# Create csv file in seed for kommun and län

In [ ]:



df_trafikanalys = df_trafikanalys[~df_trafikanalys["kommun"].astype(str).str.strip().str.contains(r"Okänd kommun|Totalt", case= False, regex= True, na= False)]
df_trafikanalys = df_trafikanalys[~df_trafikanalys["l_n"].astype(str).str.strip().str.contains(r"Okänt län|Totalt",case= False, regex= True, na=False)]



In [ ]:

# Create csv file in seeds
path = Path().cwd().parent/"data_transform/seeds"

df_kommun_lan = (df_trafikanalys[["kommun", "l_n"]].drop_duplicates())

df_kommun_lan.to_csv(path/"municipality_lan.csv", index=False, encoding= "utf-8")


In [ ]:
len(df_kommun_lan["l_n"].unique())

# Exploration marts

In [ ]:
import duckdb
from pathlib import Path


PATH_DUCKDB = Path.cwd().parent / "data_warehouse" / "data.duckdb"
    
with duckdb.connect(PATH_DUCKDB, read_only=True) as conn:
    
    query = 'SELECT * FROM marts.nr_charger;'

    curser = conn.execute(query=query)
    
    df = curser.fetchdf()
    
    df

In [ ]:
len(df)

In [ ]:
df

In [ ]:
county = df.groupby("county")["antal_ladd_stationer"].sum()

In [ ]:
county.loc["stockholm"]

In [ ]:
import duckdb
from pathlib import Path


PATH_DUCKDB = Path.cwd().parent / "data_warehouse" / "data.duckdb"
    
with duckdb.connect(PATH_DUCKDB, read_only=True) as conn:
    
    query = 'SELECT * FROM marts.infrastructur;'

    curser = conn.execute(query=query)
    
    df_infrastructur = curser.fetchdf()
    
df_infrastructur

## Graph

In [ ]:
import plotly_express as px
from backend.data_processing import query_analytics


df = query_analytics("nr_charger")

data = df[df["county"] == "stockholm"].sort_values(
    by="antal_ladd_stationer", ascending=True
)

fig = px.bar(
    data,
    x="antal_ladd_stationer",
    y="municipality",
    title="Antal laddstationer för varje kommun",
    orientation="h",
    template="simple_white",
    width=500,
)

fig.update_layout(height=35 * len(data) + 150,margin= dict(l=0, r=0, t=40, b=0),showlegend = False)
fig.update_yaxes(ticks="", minor=dict(ticks=""))
fig.show()

# Graph with snowflake

## connect with database

In [ ]:
import os 
from pathlib import Path
from dotenv import load_dotenv
import snowflake.connector
import pandas as pd



    
load_dotenv()

with snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE") )as conn:
    
    query = 'SELECT * FROM marts.nr_charger;'

        
    cur = conn.cursor()   
    data =cur.execute(query)
    
    df = data.fetch_pandas_all()
    
df 

In [ ]:
import os 
from dotenv import load_dotenv
import snowflake.connector
import pandas as pd



    
load_dotenv()

with snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE") )as conn:
    
    query = 'SELECT * FROM marts.nr_charger;'

        
    cur = conn.cursor()   
    data =cur.execute(query)
    
    df_infra = data.fetch_pandas_all()
    
df_infra

In [ ]:

df_infra.query("COUNTY == 'stockholm'")





stockholm_df = (
    df_infra.query("COUNTY == 'stockholm'")[["MUNICIPALITY", "LADDPUNKTER", "TOTAL_VEHICLE", "ANTAL_SNABB_LADD_STATIONER", "ANTAL_LADD_STATIONER"]]
    )

stockholm_df["ELBIL_PER_LADDPUNKT"] = stockholm_df["TOTAL_VEHICLE"]/ stockholm_df["LADDPUNKTER"]
stockholm_df["PROCENT_SNABB"] = round(stockholm_df["ANTAL_SNABB_LADD_STATIONER"] / stockholm_df["ANTAL_LADD_STATIONER"] * 100, 2)


stockholm_df

 


In [ ]:
import plotly_express as px



ticksval_x = [1,2,5,10,20,50,100,200,500,1000,2000,5000,10000]
ticksval_y = [10, 20, 50, 100]

stockholm_df["PROCENT_SNABB_SIZE"] = stockholm_df["PROCENT_SNABB"].clip(lower= 1)

fig = px.scatter(
    stockholm_df,
    x=stockholm_df["LADDPUNKTER"],
    y=stockholm_df["ELBIL_PER_LADDPUNKT"],
    hover_name=stockholm_df["MUNICIPALITY"],
    size= stockholm_df["PROCENT_SNABB_SIZE"],
    color= stockholm_df["MUNICIPALITY"],
    size_max=25,
    opacity=0.6,
    template="simple_white"
)



# Add line where elbil per laddpunkt is 10. 
fig.add_hline(y=10, line_width= 2, line_color= "#02010F", opacity=0.6)


fig.update_traces(marker=dict(line=dict(width=0)))

# update y- and x-axel and its ticks
fig.update_xaxes(type="log", tickmode= "array",  tickvals= ticksval_x, ticktext= [str(v) for v in ticksval_x], ticks= "", title_text= "")
fig.update_xaxes(showline= True, linewidth= 1, linecolor="rgba(0,0,0,0.3)")
fig.update_yaxes(type= "log", tickmode= "array", tickvals= ticksval_y, ticktext= [str(v) for v in ticksval_y], ticks= "", title_text= "")
fig.update_yaxes(showline= True, linewidth=1, linecolor="rgba(0,0,0,0.3)")


# Give titel to y-axel
fig.add_annotation(
    text= "ELBILAR PER LADDPUNKT",
    font= dict(
        size= 13,
        color= "black",
        family= "Ariel"
    ),
    xref= "paper",
    yref= "paper",
    x= -0.0605,
    y= 1.10,
    xanchor="center",
    yanchor= "top",
    opacity= 1,
    showarrow=False
)

# Give title to x-axel
fig.add_annotation(
    text= "LADDPUNKTER PER KOMMUN",
    font= dict(
        size= 15,
        color="black",
        family= "Ariel"
        ), 
    xref= "paper",
    yref= "paper",
    x= 0.1,
    y= -0.18,
    opacity=1,
    xanchor="center",
    yanchor= "bottom",
    showarrow=False
)


# update title 
fig.update_layout(
    title={
        "text": "Elbil per laddpunkt.<br>" 
        "Cirkelns storlek bestäms över hur stor procent kommuner snabb laddare.",
        "y": 0.95,
        "x": 0.11,
        "xanchor": "left",
        "yanchor": "top"
        
    },
    margin= dict(l= 155, b= 70, t= 100)
)



fig.show()

## Story telling graf


In [2]:
import os 
from dotenv import load_dotenv
import snowflake.connector
import pandas as pd



    
load_dotenv()

with snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA"),
    role=os.getenv("SNOWFLAKE_ROLE") )as conn:
    
    query = 'SELECT * FROM marts.nr_charger;'

        
    cur = conn.cursor()   
    data =cur.execute(query)
    
    df_infra = data.fetch_pandas_all()
    
df_infra

,ANTAL_LADD_STATIONER,ANTAL_OPERATOR,LADDPUNKTER,ANTAL_SNABB_LADD_STATIONER,TOTAL_VEHICLE,MUNICIPALITY,COUNTY
0,89,25,578,18,12703,jönköping,jönköping
1,11,9,43,5,1057,sala,västmanland
2,15,12,87,4,3391,motala,östergötland
3,56,19,858,14,9016,mölndal,västra götaland
4,98,25,849,26,13062,linköping,östergötland
...,...,...,...,...,...,...,...
284,5,3,30,1,416,hagfors,värmland
285,11,1,22,0,645,svenljunga,västra götaland
286,9,5,45,2,712,tomelilla,skåne
287,18,9,203,2,4160,upplands väsby,stockholm


In [3]:
import plotly_express as px



df = df_infra
df_county = df.query(f"COUNTY == 'stockholm'")[["MUNICIPALITY", "LADDPUNKTER", "TOTAL_VEHICLE", "ANTAL_SNABB_LADD_STATIONER", "ANTAL_LADD_STATIONER"]]

df_county["ELBIL_PER_LADDPUNKT"] = df_county["TOTAL_VEHICLE"]/ df_county["LADDPUNKTER"]
df_county["PROCENT_SNABB"] = round(df_county["ANTAL_SNABB_LADD_STATIONER"] / df_county["ANTAL_LADD_STATIONER"] * 100, 2)

ticksval_x = [1,2,5,10,20,50,100,200,500,1000,2000,5000,10000]
ticksval_y = [5, 10, 15, 20, 30, 50, 100]


fig = px.scatter(
    df_county,
    x="LADDPUNKTER",
    y="ELBIL_PER_LADDPUNKT",
    hover_name="MUNICIPALITY",
    color= "MUNICIPALITY",
    size_max=30,
    opacity=0.6,
    template="plotly_white",
    hover_data= {
        "LADDPUNKTER": ":.0f",
        "ELBIL_PER_LADDPUNKT": ":.1f",
        "PROCENT_SNABB": ":.1f",
        "MUNICIPALITY": False
    },
    labels= {
        "LADDPUNKTER": "Laddpunkter",
        "ELBIL_PER_LADDPUNKT": "Elbilar per laddpunkter",
        "PROCENT_SNABB": "Andel sanbbladdare (%)",
        "MUNICIPALITY": "Kommuner"
    }
)



# Add line where elbil per laddpunkt is 10. 
fig.add_hline(y=10, line_width= 2, line_color= "#02010F", opacity=0.6)


fig.update_traces(marker=dict(line=dict(width=0)))

# update y- and x-axel
fig.update_xaxes(type="log", tickmode= "array",  tickvals= ticksval_x, ticktext= [str(v) for v in ticksval_x], ticks= "", title_text= "")
fig.update_xaxes(showline= True, linewidth= 1, linecolor="rgba(0,0,0,0.3)")
fig.update_yaxes(type= "log", tickmode= "array", tickvals= ticksval_y, ticktext= [str(v) for v in ticksval_y], ticks= "",title_text= "")
fig.update_yaxes(showline= True, linewidth=1, linecolor="rgba(0,0,0,0.3)")



# Give titel to y-axel
fig.add_annotation(
    text= "ELBILAR PER<br>""LADDPUNKT",
    font= dict(
        size= 13,
        color= "black",
        family= "Ariel, sans-serif"
    ),
    xref= "paper",
    yref= "paper",
    x= 0.0,
    y= 1,
    xanchor="left",
    yanchor= "top",
    xshift= -110,
    yshift= 40,
    opacity= 0.8,
    showarrow=False
)

# Give title to x-axel
fig.add_annotation(
    text= "ANTAL LADDPUNKTER",
    font= dict(
        size= 13,
        color="black",
        family= "Ariel, sans-serif"
        ), 
    xref= "paper",
    yref= "paper",
    x= 0.0,
    y= 0.0,
    opacity=0.8,
    xanchor="left",
    yanchor= "top",
    yshift= -30,
    showarrow=False
)



# update title 
fig.update_layout(
    title=dict(
        text= "Horisontell linje visar EU:s referenspunkt för cirka 10 elbilar per laddpunkt.<br>",
        y= 0.90,
        x= 0.0,
        xanchor= "left",
        yanchor= "top",
        xref= "paper",
        pad=dict(l=0),
        font= dict(
            size= 17,
            family= "Ariel, sans-serif",
            color= "#292B2F" 
        )
        
    ),
    margin= dict(l= 110, b= 70, t= 100),
)

fig.update_traces(
    marker=dict(
        size= 14,
        opacity=1,
    )
                  )

fig.show()
fig.write_image("../data/scatter_plot.png", width= 2000, height= 700, scale= 2)
